In [1]:
import json
import os
from datetime import datetime, timedelta
from functools import lru_cache
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import plotly.express as px
import matplotlib.pyplot as plt

# --- Configuration ---
plt.style.use('seaborn-v0_8')
px.defaults.template = 'plotly_white'

CACHE_DIR = Path('cache')
CACHE_DIR.mkdir(exist_ok=True)

EIA_API_KEY = os.getenv('EIA_API_KEY')
EIA_BASE_URL = 'https://api.eia.gov/v2/electricity/'

# Codes for "Carbon-Free" energy sources
GREEN_CODES = {'SUN', 'WND', 'WAT', 'GEO', 'NUC'}

## EIA Hourly Demand Fetching


In [2]:
# --- 1. Fetch Real Hourly Demand ---
@lru_cache(maxsize=None)
def fetch_eia_hourly(region: str) -> pd.DataFrame:
    """Fetch hourly demand (MW)."""
    url = EIA_BASE_URL + 'rto/region-data/data/'
    end = datetime.utcnow()
    start = end - timedelta(days=90)
    
    params = {
        'api_key': EIA_API_KEY,
        'data[0]': 'value',
        'facets[respondent][]': region,
        'frequency': 'hourly',
        'start': start.strftime('%Y-%m-%dT%H'),
        'end': end.strftime('%Y-%m-%dT%H'),
        'sort[0][column]': 'period',
        'sort[0][direction]': 'desc',
        'length': 5000,
    }
    
    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json().get('response', {}).get('data', [])
        df = pd.DataFrame(data)
        if not df.empty:
            df['datetime'] = pd.to_datetime(df['period'])
            df['demand_MW'] = df['value'].astype(float)
            return df.sort_values('datetime')
        return pd.DataFrame()
    except Exception as e:
        print(f"⚠️ Demand fetch failed for {region}: {e}")
        return pd.DataFrame()

# --- 2. Fetch Real Fuel Mix ---
@lru_cache(maxsize=None)
def fetch_eia_fuelmix(region: str) -> float:
    """Fetch Carbon-Free Energy % (0-100)."""
    url = EIA_BASE_URL + 'rto/fuel-type-data/data/'
    end = datetime.utcnow()
    start = end - timedelta(hours=24)
    
    params = {
        'api_key': EIA_API_KEY,
        'data[0]': 'value',
        'facets[respondent][]': region,
        'frequency': 'hourly',
        'start': start.strftime('%Y-%m-%dT%H'),
        'end': end.strftime('%Y-%m-%dT%H'),
        'length': 500
    }
    
    try:
        r = requests.get(url, params=params, timeout=30)
        data = r.json().get('response', {}).get('data', [])
        if not data: return float('nan')
            
        df = pd.DataFrame(data)
        df['value'] = df['value'].astype(float)
        
        # Get most recent hour
        latest_time = df['period'].max()
        current_mix = df[df['period'] == latest_time]
        
        total = current_mix['value'].sum()
        if total == 0: return 0.0
            
        clean = current_mix[current_mix['fueltype'].isin(GREEN_CODES)]['value'].sum()
        return (clean / total) * 100.0
    except:
        return float('nan')

# --- 3. Fetch Real Price (NEW) ---
@lru_cache(maxsize=None)
def fetch_eia_price(region: str) -> float:
    """Fetch latest Wholesale Price ($/MWh)."""
    url = EIA_BASE_URL + 'wholesale-markets-data/data/'
    end = datetime.utcnow()
    start = end - timedelta(hours=24)
    
    # Note: Mapping Regions to Hubs is complex. 
    # For this prototype, we will use a heuristic: 
    # If specific price data is missing, we fallback to a calculated proxy later.
    # This generic call attempts to get any LMP data for the region.
    params = {
        'api_key': EIA_API_KEY,
        'data[0]': 'value',
        'facets[respondent][]': region,
        'frequency': 'hourly',
        'sort[0][column]': 'period',
        'sort[0][direction]': 'desc',
        'length': 10
    }
    try:
        r = requests.get(url, params=params, timeout=5)
        data = r.json().get('response', {}).get('data', [])
        if data:
            # return average of the last few reported prices
            vals = [float(x['value']) for x in data if x['value']]
            return np.mean(vals) if vals else float('nan')
    except:
        pass
    return float('nan')

# --- 4. Fetch Weather ---
@lru_cache(maxsize=None)
def fetch_temperature(lat, lon):
    try:
        url = 'https://api.open-meteo.com/v1/forecast'
        params = {'latitude': lat, 'longitude': lon, 'daily': 'temperature_2m_mean', 'past_days': 60}
        r = requests.get(url, params=params, timeout=10)
        temps = r.json().get('daily', {}).get('temperature_2m_mean', [])
        return np.mean(temps) if temps else np.nan
    except:
        return np.nan

## Compute Datacenter Scores

In [3]:

# --- MAIN EXECUTION ---
region_coords = {
    'CAL': (36.5, -119.5), 'CAR': (35.5, -80.0), 'CENT': (38.5, -94.5),
    'FLA': (28.0, -82.0), 'MIDA': (39.0, -77.0), 'MIDW': (42.0, -89.0),
    'NE': (42.5, -72.5), 'NY': (42.9, -75.3), 'NW': (45.5, -120.5),
    'SE': (33.0, -84.0), 'SW': (36.0, -111.5), 'TEN': (36.0, -86.0),
    'TEX': (31.0, -99.0),
}

records = []
print("🚀 Fetching Real Data...")

for region, (lat, lon) in region_coords.items():
    # A. Fetch Raw Data
    df_demand = fetch_eia_hourly(region)
    raw_renew = fetch_eia_fuelmix(region)
    raw_price = fetch_eia_price(region)
    raw_temp = fetch_temperature(lat, lon)
    
    # B. Compute Derived Metrics
    if not df_demand.empty:
        raw_load = df_demand['demand_MW'].iloc[-1]
        # Volatility (Standard Deviation of last 24h)
        raw_volatility = df_demand['demand_MW'].tail(24).std()
        # Peak Forecast (Simple max of last 30 days as a proxy for capacity)
        raw_peak = df_demand['demand_MW'].max()
    else:
        raw_load, raw_volatility, raw_peak = np.nan, np.nan, np.nan

    # Fallback logic if Price API returns nothing (common for some regions)
    # We proxy price using Load Stress (High Load = High Price)
    if np.isnan(raw_price) and not np.isnan(raw_load):
        raw_price = (raw_load / 1000) * 2.5 # Rough heuristic $2.50 per GW

    records.append({
        'region': region,
        'lat': lat, 'lon': lon,
        'raw_price': raw_price,
        'raw_load': raw_load,
        'raw_volatility': raw_volatility,
        'raw_peak': raw_peak,
        'raw_renew': raw_renew,
        'raw_temp': raw_temp
    })

# --- DATAFRAME CONSTRUCTION ---
dc_df = pd.DataFrame(records)

# 1. Fill Missing Data (Mean Imputation)
dc_df = dc_df.fillna(dc_df.mean(numeric_only=True))

# 2. Normalize Columns (Store as 'n_column')
# We normalize so 0 is "Bad" and 1 is "Good"
# Price: Lower is better -> 1 - norm
# Load: Lower is better -> 1 - norm
# Volatility: Lower is better -> 1 - norm
# Temp: Lower is better -> 1 - norm
# Renewables: Higher is better -> norm

def normalize(series, invert=False):
    min_v, max_v = series.min(), series.max()
    if max_v == min_v: return 0.5
    norm = (series - min_v) / (max_v - min_v)
    return (1 - norm) if invert else norm

dc_df['n_price'] = normalize(dc_df['raw_price'], invert=True)
dc_df['n_load'] = normalize(dc_df['raw_load'], invert=True)
dc_df['n_volatility'] = normalize(dc_df['raw_volatility'], invert=True)
dc_df['n_temp'] = normalize(dc_df['raw_temp'], invert=True)
dc_df['n_renew'] = normalize(dc_df['raw_renew'], invert=False) # Higher is good

# 3. Calculate Scores
# Profitability (40%): Price, Load, Volatility
dc_df['profitability'] = (
    0.40 * dc_df['n_price'] +
    0.30 * dc_df['n_load'] +
    0.30 * dc_df['n_volatility']
)

# Sustainability (60%): Renewables, Temp
dc_df['sustainability'] = (
    0.70 * dc_df['n_renew'] + 
    0.30 * dc_df['n_temp']
)

# Final Score
dc_df['dc_score'] = 0.40 * dc_df['profitability'] + 0.60 * dc_df['sustainability']

# Sort
dc_df_final = dc_df.sort_values('dc_score', ascending=False).reset_index(drop=True)

# Save EVERYTHING
dc_df_final.to_csv('datacenter_scores_real.csv', index=False)

🚀 Fetching Real Data...


/var/folders/h3/f3_68x516pj6d7h2119r5st00000gn/T/ipykernel_15202/1253555189.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.utcnow()
/var/folders/h3/f3_68x516pj6d7h2119r5st00000gn/T/ipykernel_15202/1253555189.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.utcnow()
/var/folders/h3/f3_68x516pj6d7h2119r5st00000gn/T/ipykernel_15202/1253555189.py:78: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.utcnow()
/var/folders/h3/f3_68x516pj6d7h2119r5st00000gn/T/ipykernel_15202/

In [4]:
dc_df_final

,region,lat,lon,raw_price,raw_load,raw_volatility,raw_peak,raw_renew,raw_temp,n_price,n_load,n_volatility,n_temp,n_renew,profitability,sustainability,dc_score
0,NY,42.9,-75.3,47.0075,18803.0,651.338754,20325.0,51.350038,5.800000,0.919927,0.919927,0.936484,1.000000,0.804842,0.924894,0.863389,0.887991
1,CAR,35.5,-80.0,55.9950,22398.0,688.416472,35825.0,60.187159,13.795522,0.877463,0.877463,0.926423,0.483513,1.000000,0.892151,0.845054,0.863893
2,TEN,36.0,-86.0,44.8175,17927.0,417.266953,26174.0,50.679782,13.898507,0.930274,0.930274,1.000000,0.476861,0.790040,0.951192,0.696086,0.798129
3,NW,45.5,-120.5,114.7850,45914.0,2068.570205,49728.0,55.439846,8.534328,0.599693,0.599693,0.551913,0.823371,0.895161,0.585359,0.873624,0.758318
4,NE,42.5,-72.5,38.5175,15407.0,640.962908,16897.0,32.106455,7.720896,0.960040,0.960040,0.939299,0.875916,0.379869,0.953818,0.528683,0.698737
5,SW,36.0,-111.5,30.0600,12024.0,835.919748,18625.0,32.464061,12.017910,1.000000,1.000000,0.886397,0.598342,0.387767,0.965919,0.450939,0.656931
6,SE,33.0,-84.0,65.7675,26307.0,1093.603683,38910.0,35.679350,15.467164,0.831290,0.831290,0.816473,0.375530,0.458773,0.826845,0.433800,0.591018
7,CENT,38.5,-94.5,84.3050,33722.0,1715.223142,44968.0,28.651758,13.246269,0.743704,0.743704,0.647795,0.518993,0.303576,0.714931,0.368201,0.506893
8,CAL,36.5,-119.5,73.7225,29489.0,2718.355284,38450.0,28.667029,15.983582,0.793704,0.793704,0.375591,0.342171,0.303913,0.668270,0.315391,0.456543
9,MIDA,39.0,-77.0,241.7100,96684.0,1695.349482,111773.0,38.694256,11.789552,0.000000,0.000000,0.653187,0.613093,0.525353,0.195956,0.551675,0.409388
